# HD BITCOIN WALLET

This project aims to implement and Hierarchical Deterministic(HD) wallet using the bitcoin standards BIP-32 
and BIP-39 by following the guidelines from the book Mastering Bitcoin. An HD wallet allows to derive many keys 
from a single seed stored in a tree structure. With respect to non deterministic wallets, where keys are generated 
at random and have no connection with one another, the HD wallet allows for simpler managment and doesn't require
to keep copies of all the keys since they can all be derived from the seed. Furthermore HD wallets allow to derive 
public keys without the need to access to the associated private keys which guarantees additional security in 
insecure scenarios.

<img src="CryptoProject/pic.png" style ="width:500px; height: 300px">

In [1]:
"""Now we import all the required libraries"""

from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac



In [2]:
"""here i create a class to store data in a tree like structure, every node has a parent, a private key,
a chain code and a list of children"""

class TreeNode():
    
    def __init__(self,parent,pvt_key,pub_key,chain_code): #every node has a parent, the master will have None as input
        self.parent=parent
        self.pvt_key=pvt_key 
        self.pub_key=pub_key
        self.chain_code =chain_code
        self.children = []
    

Now we need to follow the seed generation procedure as described by BIP-39. The whole process is performed in a few commands thanks to the Mnemonic library.The concept is of generating a random entropy which is then mapped to some words (mnemonic words), which can be used to generate the actual seed.The idea of using mnemonic words comes from the fact that they are much simpler to remember than the seed itself and can be used to unambigously retrive all of the keys in the wallet, allowing for a backup in case of loss or destruction. The whole procedure is explained in the following images
<img src="pic2.png" style ="width:700px; height: 500px">
<img src="pic3.png" style ="width:700px; height: 500px">


In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght): #strenght refers to the lenght in bits of the entropy to generate 
                                                 #passphrase is an additional value used to append to the salt for PBKDF2
    
   mnemo = Mnemonic(language)

   words = mnemo.generate(strength=strenght) #this command performs steps 1 to 5 and saves the words in a vavariable
    
   seed = mnemo.to_seed(words, passphrase=passphrase)#here we retrieve the seed from the words (step 7 to 9)

   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
    
   print("Words= " + words)   

   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
    
   return seed 
   


In [4]:
#Here we simply test our function and check the values generated 
seed=generate_seed("english","test",128)

Entropy=b'9c0dc86c55b40e57cc7b376d84200424'
Words= orchard hour brand problem domain cloud cradle group horn canal absurd embark
Seed = b'447efd4feb58011d806fd2724156dd4b2e06d5c2fddec71b3d2348fd4ac93ab46cfa8dc6c4cd2eebfe3f4935f499378b376724d102b354d84a79b0c4c832c0e5'


Now that we have the seed it's time to generate the master. We make use of HMAC-SHA512 in order to have a fixed lenght output and to protect the seed from some possible attacks(to this end the hash is one way).It is worth noticing that the public key is then generated starting from the private key using ellipric curve theory, where public key = G*private key and G is the generator point of the curve. As the theory suggests, the public key is a pair of coordinates x and y, so to obtain the actual public key I take the first two bytes of the y coordinate and combine them with all the ones from the x axis. The following picture explains the process:
<img src="pic4.png" style ="width:600px; height: 400px">

In [5]:
"""In this section I derive from the seed the master private key and the master chain code, 
the private key is the one used to have control over your bank account represented by the public key. 
The bitcoin address is generated/corresponds from/to the public key"""

def get_master_info(seed):
    
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest() #here we perform HMAC-SHA512 on the root seed
    
    master_pvt_key=master_hmac[:64] #we pick the first 256 bits (256/4=64 since we are working in hex) 
    
    master_chain_code=master_hmac[64:] #we extract the chain code using a similar logic 
    
    #Now we need to derive the public key for our master and I have used fastecdsa for a quick result
    
    pvt=int(master_pvt_key, 16) #some conversion for compliance with the function 
    
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1) # we generate the public key using the secp256k1 elliptic curve 
    
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #for the serialization of the public key i used the hints at: https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    
    master=TreeNode(None,master_pvt_key,pub_key,master_chain_code) #I create a TreeNode instance and pass as parent None, since we are creating the root 
    
    return master


In [6]:
master_info=get_master_info(seed)
master_info.pvt_key


'17c1a782153e996f008fa69cd480c275b55f1ce44cc95df370cb82f999fcb235'

Now that we have the master we have to create the children.In order to do so we need the parent's public or private key, the child index and the parent chain code. The last one is needed in order to introduce deterministic random data so that something more is required than the index and the child key to get to the siblings of a determined node. 
We have two types of children:
1) regular children which are derived using the public address of the parent
<img src="pic6.png" style ="width:600px; height: 400px">
2) hardened children which are derived from the private address of the parent
<img src="pic5.png" style ="width:600px; height: 400px">
Hardened children are particularly useful when we are with public extended keys where we need to deal with the possibility of having a children private key leaked. Since from the extended public address we can derive the chain code it is possible using the two to get the private keys of other children or even worse the one of the master. So we want to break the relationship between parent public key and child chain code.In this way the public key doesn't give any useful info on the other children private keys.In this way we can create extended public keys which are not vulnerable.


In [7]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex, num represents the number od children we want to create 
  
    for i in range(0,num): 
        
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:]) #here I concatenate all the values that should be hashed 
        
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest() #Here I perform HMAC-SHA512 on the input string
        
        pvt_key=child_info[:64] #similarly as the master we extract the private key
        
        chain_code=child_info[64:]
        
        pvt=int(pvt_key, 16) #compliance transformations 
        
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #same as master 
        
        child=TreeNode(parent.pvt_key,pvt_key,pub_key,chain_code) #I create the TreeNode entity
        
        parent.children.append(child) #I append the new entity to the list of children of the parent 

In [8]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:]) #here I concatenate all the values that should be hashed 
        
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest() #Here I perform HMAC-SHA512 on the input string
        
        pvt_key=child_info[:64] #similarly as the master we extract the private key
        
        chain_code=child_info[64:]
        
        pvt=int(pvt_key, 16) #compliance transformations 
        
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #same as master 
        
        child=TreeNode(parent.pvt_key,pvt_key,pub_key,chain_code) #I create the TreeNode entity
        
        parent.children.append(child) #I append the new entity to the list of children of the parent 
            

Now we are ready to create some children, as an example I create two regular children and and one hardened children. 

In [9]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")

Private Key ec5fbff946aac4910aa8c35b50a71f0023dceaa70a138382da04af5487b5de9a
Public Key 7f473d0d6132f6c9749dc28a91ed740fd7b8c962ef31a101d284f538f2afc1aa04
Chain Code 305fff2f5361f1a2d06219f096fc2316c48b12144f3f6b1cb42e617914749217
-------------------------------------------------------------------------
Private Key f8c351852587394a76bcb45d39ca2194d5f4bc8f028ab25c3a15e3db46f167a2
Public Key e9d8f84125e00e115df22491991b162ad231af0fa6756bed124f5d303738774947
Chain Code f08c02cd8b9c5c6eb3b5c758bec6e2b974d49761df5f3d8ffe9cc65595573f11
-------------------------------------------------------------------------
Private Key baa077b16f122417d1ca4741972eb759d772c465b9c432d54982b4f5c61f14e5
Public Key af4ce8802ceea99bbb477cd0e5055809a33de387343b8a4445992817f54d512bee
Chain Code d1d9fa0cbe45a5323150ba921e357a002b5d2401bd7040b30e88a99e4fb806c8
-------------------------------------------------------------------------


Now that we have some children let's create some grandkids. Each child will have one hardened children and two regular children. The result will show a total of nine nodes (3x3=9)

In [10]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key 631467fc94a79fbbf369bf1cef0aa9c1870c9b01f8b7984aae17408b6b94eb1a
Public Key 9e93184bf7a4a22ee65fc624dcaf562b7d6ae2c7c4d9c82e769cc302f4d94b69d
Chain Code a30a4d219314acfcd48529362b866b41908146753bcddde25074410bc3a8cc43
-------------------------------------------------------------------------
Private Key 41a4cf4ac2db3a8c6e4e5b5df61483bd02c47c251277f087efd834ee321f6b1f
Public Key 54c8ec205337f089deb0a1720e508eede6de6cbd9a99a93cc4dac06ff4eebcf4be
Chain Code 8943304f01d5516a72671dbd21fdd6a04601a85c520d7f1695e3d32a85540091
-------------------------------------------------------------------------
Private Key b4d7c4b065ec322306dd197123968b4f50be9c4bff7d3eee05406c4b2e026645
Public Key b6105f2e7f24e9cb1d03d284ce6ff4ecc2be03bb1d2a16bf99ff033620b1b1ad47
Chain Code 4a5e89568db56a551bcc6b1e2c9a38ef9e48a9b86f1bb533b4bdbe83f4b6d82b
-------------------------------------------------------------------------
Private Key e84c775d7dee36f15e931dffd10568d0c22e3478559ee1fcb297913602c9c48a
Public Ke